In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint


# Import API key
from api_keys2 import g_key

# Import city finding library for lat & long.
from citipy import citipy



In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'city_weather.csv')
city_df = pd.read_csv(csv_file)
city_df

,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed
0,3939761,Hualmay,PE,-11.0964,-77.6139,71.35,80,100,10.45
1,2017155,Saskylakh,RU,71.9167,114.0833,-31.54,100,6,2.04
2,2129766,Kamiiso,JP,41.8167,140.6500,32.38,86,75,29.93
3,3883457,Lebu,CL,-37.6167,-73.6500,56.91,90,0,7.52
4,2110227,Butaritari,KI,3.0707,172.7902,82.13,75,54,15.97
...,...,...,...,...,...,...,...,...,...
460,6111867,Port Hawkesbury,CA,45.6169,-61.3485,9.77,80,100,16.11
461,1715010,Dicamay,PH,16.7561,122.0037,72.91,89,95,2.01
462,1140025,Herat Province,AF,34.5000,62.0000,32.65,77,0,9.55
463,2138285,Tadine,NC,-21.5500,167.8833,81.27,81,100,15.55


In [3]:
gmaps.configure(api_key = g_key)

#Create coordinate pairs from city dataframe & import humidity values
coordinates = city_df[['Latitude', 'Longitude']]
humidity = city_df['Humidity']

#Display it (?lol)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3

#Add heat layer to the figure
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
idealvacay_df = city_df[(city_df['Temperature'] <= 75) & (city_df['Temperature'] >= 60)  
                       & (city_df['Clouds'] >= 45) & (city_df['Humidity'] <= 60)]
idealvacay_df



,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed
7,2156643,Mount Gambier,AU,-37.8333,140.7667,69.76,45,100,3.69
15,2381334,Atar,MR,20.5169,-13.0499,66.22,21,96,9.42
42,2191562,Dunedin,NZ,-45.8742,170.5036,60.80,56,100,8.01
125,2450173,Taoudenni,ML,22.6783,-3.9836,65.01,25,100,12.19
174,233093,Kaabong,UG,3.4836,34.1492,67.86,47,100,3.06
261,2381972,Adrar,MR,20.5022,-10.0711,63.27,23,99,14.29
263,343292,Āsosa,ET,10.0667,34.5333,66.87,27,77,3.74
268,64814,Bandarbeyla,SO,9.4942,50.8122,73.08,60,71,13.29
300,2386756,Gamboula,CF,4.1178,15.1393,70.54,33,77,1.52


In [5]:
#Adding new column to store returned values for nearest hotel
idealvacay_df.insert(loc=9,column="Nearby Hotel",value="")
idealvacay_df

#Tried df['column name'] = "" but returned an error that implied it would be unstable?

,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed,Nearby Hotel
7,2156643,Mount Gambier,AU,-37.8333,140.7667,69.76,45,100,3.69,
15,2381334,Atar,MR,20.5169,-13.0499,66.22,21,96,9.42,
42,2191562,Dunedin,NZ,-45.8742,170.5036,60.80,56,100,8.01,
125,2450173,Taoudenni,ML,22.6783,-3.9836,65.01,25,100,12.19,
174,233093,Kaabong,UG,3.4836,34.1492,67.86,47,100,3.06,
261,2381972,Adrar,MR,20.5022,-10.0711,63.27,23,99,14.29,
263,343292,Āsosa,ET,10.0667,34.5333,66.87,27,77,3.74,
268,64814,Bandarbeyla,SO,9.4942,50.8122,73.08,60,71,13.29,
300,2386756,Gamboula,CF,4.1178,15.1393,70.54,33,77,1.52,


In [6]:
for index, row in idealvacay_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        params['location'] = f"{row['Latitude']}, {row['Longitude']}"

        hotel_data = requests.get(base_url, params=params).json()
#         pprint(hotel_data)
        
        idealvacay_df.loc[index, "Nearby Hotel"] = hotel_data["results"][0]["name"]
        idealvacay_df.loc[index, "Hotel Lat"] = hotel_data["results"][0]["geometry"]["location"]["lat"]
        idealvacay_df.loc[index, "Hotel Lng"] = hotel_data["results"][0]["geometry"]["location"]["lng"]
        
    except IndexError:
        
        idealvacay_df.loc[index, "Nearby Hotel"] = "NaN"
        
idealvacay_df



/Users/joshbarrett/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/joshbarrett/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed,Nearby Hotel,Hotel Lat,Hotel Lng
7,2156643,Mount Gambier,AU,-37.8333,140.7667,69.76,45,100,3.69,Park Hotel,-37.825862,140.772144
15,2381334,Atar,MR,20.5169,-13.0499,66.22,21,96,9.42,Bab Sahara,20.518977,-13.062143
42,2191562,Dunedin,NZ,-45.8742,170.5036,60.80,56,100,8.01,The Law Courts Hotel,-45.875095,170.506645
125,2450173,Taoudenni,ML,22.6783,-3.9836,65.01,25,100,12.19,NaN,NaN,NaN
174,233093,Kaabong,UG,3.4836,34.1492,67.86,47,100,3.06,Range Lands Hotel,3.512483,34.133675
261,2381972,Adrar,MR,20.5022,-10.0711,63.27,23,99,14.29,NaN,NaN,NaN
263,343292,Āsosa,ET,10.0667,34.5333,66.87,27,77,3.74,Bamboo Paradise Hotel,10.059722,34.544722
268,64814,Bandarbeyla,SO,9.4942,50.8122,73.08,60,71,13.29,JABIR HOTEL,9.497559,50.810586
300,2386756,Gamboula,CF,4.1178,15.1393,70.54,33,77,1.52,NaN,NaN,NaN


In [7]:
#In order to create markers for the exact hotel location rather than the city location, NaN values need to be dropped.
#The code in the cell below returns an error for NaN values, so for Kyabe, where no hotel was found, I need to drop
#this row. Plus, why would I go if there's nowhere to stay?

idealvacay_df.dropna(how='any',inplace=True)
idealvacay_df

/Users/joshbarrett/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,City ID,City,Country,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed,Nearby Hotel,Hotel Lat,Hotel Lng
7,2156643,Mount Gambier,AU,-37.8333,140.7667,69.76,45,100,3.69,Park Hotel,-37.825862,140.772144
15,2381334,Atar,MR,20.5169,-13.0499,66.22,21,96,9.42,Bab Sahara,20.518977,-13.062143
42,2191562,Dunedin,NZ,-45.8742,170.5036,60.80,56,100,8.01,The Law Courts Hotel,-45.875095,170.506645
174,233093,Kaabong,UG,3.4836,34.1492,67.86,47,100,3.06,Range Lands Hotel,3.512483,34.133675
263,343292,Āsosa,ET,10.0667,34.5333,66.87,27,77,3.74,Bamboo Paradise Hotel,10.059722,34.544722
268,64814,Bandarbeyla,SO,9.4942,50.8122,73.08,60,71,13.29,JABIR HOTEL,9.497559,50.810586


In [9]:
hotel_locations = idealvacay_df[['Hotel Lat','Hotel Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(hotel_locations, 
                             info_box_content=[f"The nearest hotel is: {idealvacay_df['Nearby Hotel']}" 
                                               for index, row in idealvacay_df.iterrows()])
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))